- match and replace the v1 priorities with v0.5
- also boost previously observed

In [66]:
import numpy as np
import glob
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import matplotlib.font_manager as font_manager
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
font_prop = font_manager.FontProperties(size=18)
plt.rcParams["font.family"] = "Times"
%matplotlib inline

v05_survey_cartons = glob.glob("./BHM_*_v0.5.fits")
v1_survey_cartons = glob.glob("./BHM_*_v1.fits")

print(v05_survey_cartons)
print(v1_survey_cartons)

rmV1 = Table.read(v1_survey_cartons[1], format='fits')
rmV05 = Table.read(v05_survey_cartons[5], format='fits')

#science masks 
v1_sci = (rmV1['category']=='science')
v05_sci = (rmV05['category']=='science')

['./BHM_cartons_SCVZ_survey_v0.5.fits', './BHM_cartons_ELAIS_survey_v0.5.fits', './BHM_cartons_XMMLSS_survey_v0.5.fits', './BHM_cartons_RM_survey_v0.5.fits', './BHM_cartons_COSMOS_survey_v0.5.fits', './BHM_cartons_DESCNC3_survey_v0.5.fits']
['./BHM_cartons_XMMLSS_v1.fits', './BHM_cartons_DESCNC3_v1.fits', './BHM_cartons_COSMOS_v1.fits', './BHM_cartons_RM_v1.fits']


In [67]:
#SO THIS WILL REPLACE THE THINGS IT CAN FIND AS OVERLAP, BUT THE SMALL RA AND DEC DIFFERENCES
#MEAN THAT IT WON"T CATCH EVERYTHING
place = 3
#other target (not known spec) overlap between the two designs
targ_overlap = np.array(list(set(list(zip(np.round(rmV05['ra'][v05_sci],place), np.round(rmV05['dec'][v05_sci],place)))) & set(list(zip(np.round(rmV1['ra'][v1_sci],place), np.round(rmV1['dec'][v1_sci],place))))))
print("Common from new to old?", len(targ_overlap))

# new_priorities = np.copy(rmV1['priority'])
# new_priorities[v1_sci] = 0


v05s = list(zip(np.round(rmV05['ra'][v05_sci],place), np.round(rmV05['dec'][v05_sci],place)))
v1s = list(zip(np.round(rmV1['ra'][v1_sci],place), np.round(rmV1['dec'][v1_sci],place)))

#rmV05[v05_sci][np.where(v05s == targ)[0]]

for targ in targ_overlap:
    print(targ)
    #print("potential matches:")
    test_v05_matches = rmV05["ra","dec","priority"][v05_sci][np.where(v05s == targ)[0]]
    #print(test_v05_matches)
    test_v1_matches = rmV1["ra","dec","priority"][v1_sci][np.where(v1s == targ)[0]]
    #print(test_v1_matches)
    
#     ind05 = int(input("which v05 index matches? (ZERO IND, INT ONLY)"))
#     ind1 = int(input("which v1 index matches? (ZERO IND, INT ONLY)"))
#     match1 = rmV1["ra","dec","priority"][v1_sci][np.where(v1s == targ)[0][ind1]]
#     match05 = rmV05["ra","dec","priority"][v05_sci][np.where(v05s == targ)[0][ind05]]

    ind05 = np.argmin(np.sqrt((3600 * (test_v05_matches['ra'] - targ[0]) * np.cos(targ[1]/ 57.296)) ** 2 +
                       (3600 * (test_v05_matches['dec'] - targ[1])) ** 2))
    ind1 = np.argmin(np.sqrt((3600 * (test_v1_matches['ra'] - targ[0]) * np.cos(targ[1]/ 57.296)) ** 2 +
                       (3600 * (test_v1_matches['dec'] - targ[1])) ** 2))
    
    match1 = rmV1["ra","dec","priority"][v1_sci][np.where(v1s == targ)[0][ind1]]
    match05 = rmV05["ra","dec","priority"][v05_sci][np.where(v05s == targ)[0][ind05]]
    
    print("sep-calced matches (v05 then v1):")
    print(match05)
    print(match1)
    new_prior = match05['priority']
    print("new priority:", match05['priority'])
    print('priority reset!')
    print()
    rmV1['priority'][np.where(rmV1['ra'] == match1['ra'])[0]] = match05['priority']
    
    print(rmV1['priority'][np.where(rmV1['ra'] == match1['ra'])[0]])
    print("-"*80)
    print("\n")

Common from new to old? 645
[ 52.918 -28.923]
sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
52.917583 -28.922733   1027.0
    ra       dec     priority
--------- ---------- --------
52.917553 -28.922752   1027.0
new priority: 1027.0
priority reset!

priority
--------
  1027.0
--------------------------------------------------------------------------------


[ 53.342 -27.514]
sep-calced matches (v05 then v1):
    ra       dec    priority
--------- --------- --------
53.341534 -27.51394   1047.0
    ra       dec    priority
--------- --------- --------
53.341534 -27.51394   1047.0
new priority: 1047.0
priority reset!

priority
--------
  1047.0
--------------------------------------------------------------------------------


[ 51.681 -28.37 ]
sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
51.680595 -28.369806   1500.0
   ra      dec     priority
------- ---------- --------
51.6806 -28.369812  

sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
53.022247 -28.056873   1500.0
    ra       dec    priority
--------- --------- --------
53.022232 -28.05688   1927.0
new priority: 1500.0
priority reset!

priority
--------
  1500.0
--------------------------------------------------------------------------------


[ 51.686 -28.065]
sep-calced matches (v05 then v1):
   ra       dec     priority
-------- ---------- --------
51.68647 -28.065434   1038.0
   ra       dec     priority
-------- ---------- --------
51.68647 -28.065434   1044.0
new priority: 1038.0
priority reset!

priority
--------
  1038.0
--------------------------------------------------------------------------------


[ 52.426 -28.319]
sep-calced matches (v05 then v1):
   ra       dec     priority
-------- ---------- --------
52.42622 -28.318552    901.0
   ra       dec     priority
-------- ---------- --------
52.42622 -28.318552   1037.0
new priority: 901.0
priority reset!

prio

sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
52.888844 -27.805117   2000.0
   ra       dec     priority
-------- ---------- --------
52.88885 -27.805136   1920.0
new priority: 2000.0
priority reset!

priority
--------
  2000.0
--------------------------------------------------------------------------------


[ 52.979 -28.286]
sep-calced matches (v05 then v1):
   ra       dec     priority
-------- ---------- --------
52.97909 -28.286154   1039.0
   ra       dec     priority
-------- ---------- --------
52.97909 -28.286154   1039.0
new priority: 1039.0
priority reset!

priority
--------
  1039.0
--------------------------------------------------------------------------------


[ 52.512 -27.547]
sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
52.512276 -27.546776    901.0
    ra       dec     priority
--------- ---------- --------
52.512276 -27.546776   1032.0
new priority: 901.0
priority reset!

sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
53.091805 -27.907938   1500.0
    ra       dec     priority
--------- ---------- --------
53.091805 -27.907942   1922.0
new priority: 1500.0
priority reset!

priority
--------
  1500.0
--------------------------------------------------------------------------------


[ 53.345 -27.923]
sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
53.344997 -27.923323    941.0
    ra       dec     priority
--------- ---------- --------
53.344997 -27.923323   1042.0
new priority: 941.0
priority reset!

priority
--------
   941.0
   941.0
   941.0
   941.0
--------------------------------------------------------------------------------


[ 51.916 -28.326]
sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
51.915516 -28.326365   1001.0
    ra       dec     priority
--------- ---------- --------
51.915516 -28.326365   1032.0


sep-calced matches (v05 then v1):
    ra       dec    priority
--------- --------- --------
53.075104 -27.78856   1500.0
    ra       dec     priority
--------- ---------- --------
53.075108 -27.788557   1920.0
new priority: 1500.0
priority reset!

priority
--------
  1500.0
--------------------------------------------------------------------------------


[ 53.075 -27.831]
sep-calced matches (v05 then v1):
    ra       dec    priority
--------- --------- --------
53.075176 -27.83148   1500.0
    ra       dec     priority
--------- ---------- --------
53.075176 -27.831482   1920.0
new priority: 1500.0
priority reset!

priority
--------
  1500.0
--------------------------------------------------------------------------------


[ 52.171 -28.665]
sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
52.171494 -28.664858   1037.0
    ra       dec     priority
--------- ---------- --------
52.171494 -28.664858   1037.0
new priority: 1037.0
priority re

...

sep-calced matches (v05 then v1):
    ra       dec    priority
--------- --------- --------
53.530647 -28.11937    942.0
    ra       dec    priority
--------- --------- --------
53.530647 -28.11937   1042.0
new priority: 942.0
priority reset!

priority
--------
   942.0
   942.0
   942.0
--------------------------------------------------------------------------------


[ 52.613 -27.179]
sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
52.613102 -27.178623    937.0
    ra       dec     priority
--------- ---------- --------
52.613102 -27.178623   1037.0
new priority: 937.0
priority reset!

priority
--------
   937.0
   937.0
   937.0
   937.0
   937.0
   937.0
--------------------------------------------------------------------------------


[ 52.629 -28.971]
sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
52.629105 -28.971014    942.0
    ra       dec     priority
--------- ---------- --------
5

priority
--------
  1500.0
--------------------------------------------------------------------------------


[ 52.136 -28.078]
sep-calced matches (v05 then v1):
   ra      dec     priority
------- ---------- --------
52.1365 -28.078144    947.0
   ra      dec     priority
------- ---------- --------
52.1365 -28.078144   1047.0
new priority: 947.0
priority reset!

priority
--------
   947.0
   947.0
--------------------------------------------------------------------------------


[ 52.427 -27.337]
sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
52.427322 -27.336708    901.0
    ra       dec     priority
--------- ---------- --------
52.427322 -27.336708   1032.0
new priority: 901.0
priority reset!

priority
--------
   901.0
   901.0
   901.0
   901.0
--------------------------------------------------------------------------------


[ 52.931 -28.251]
sep-calced matches (v05 then v1):
   ra       dec    priority
-------- --------- --------


sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
53.097767 -27.418127    947.0
    ra       dec    priority
--------- --------- --------
53.097763 -27.41812   1047.0
new priority: 947.0
priority reset!

priority
--------
   947.0
   947.0
--------------------------------------------------------------------------------


[ 53.351 -27.582]
sep-calced matches (v05 then v1):
   ra       dec     priority
-------- ---------- --------
53.35089 -27.582203   1500.0
    ra      dec    priority
--------- -------- --------
53.350895 -27.5822   1920.0
new priority: 1500.0
priority reset!

priority
--------
  1500.0
--------------------------------------------------------------------------------


[ 53.113 -28.023]
sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
53.113083 -28.023474   2000.0
   ra       dec     priority
-------- ---------- --------
53.11308 -28.023472   1920.0
new priority: 2000.0
priority res

sep-calced matches (v05 then v1):
   ra       dec     priority
-------- ---------- --------
53.23808 -27.759594   1500.0
    ra       dec    priority
--------- --------- --------
53.238083 -27.75959   1920.0
new priority: 1500.0
priority reset!

priority
--------
  1500.0
--------------------------------------------------------------------------------


[ 53.094 -27.425]
sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
53.094173 -27.424639   1032.0
    ra       dec     priority
--------- ---------- --------
53.094097 -27.424648   1032.0
new priority: 1032.0
priority reset!

priority
--------
  1032.0
--------------------------------------------------------------------------------


[ 53.251 -27.861]
sep-calced matches (v05 then v1):
   ra      dec     priority
------- ---------- --------
53.2508 -27.861292   1001.0
   ra       dec     priority
-------- ---------- --------
53.25078 -27.861288   1001.0
new priority: 1001.0
priority reset!

pri

sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
53.234722 -27.809444   1500.0
    ra       dec     priority
--------- ---------- --------
53.234722 -27.809444   1920.0
new priority: 1500.0
priority reset!

priority
--------
  1500.0
--------------------------------------------------------------------------------


[ 52.07  -28.458]
sep-calced matches (v05 then v1):
   ra       dec     priority
-------- ---------- --------
52.07027 -28.457544    942.0
   ra       dec    priority
-------- --------- --------
52.07027 -28.45755   1042.0
new priority: 942.0
priority reset!

priority
--------
   942.0
   942.0
--------------------------------------------------------------------------------


[ 52.859 -27.699]
sep-calced matches (v05 then v1):
    ra       dec     priority
--------- ---------- --------
52.859303 -27.698875   1051.0
   ra       dec     priority
-------- ---------- --------
52.85931 -27.698883   1051.0
new priority: 1051.0
priority 

## For some reason, it does't catch all of them. It's close. But not quite. Let's get the rest of v1 matched.

In [68]:
new_des_added = Table.read(v1_survey_cartons[1], format='fits')
old_des_remaining = Table.read(v05_survey_cartons[5], format='fits')

inds = []
inds_new = []
for targ in targ_overlap:
    zipped_locs = list(zip(np.round(old_des_remaining[v05_sci]['ra'],place), np.round(old_des_remaining[v05_sci]['dec'],place)))
    zipped_locs_new = list(zip(np.round(new_des_added[v1_sci]['ra'],place), np.round(new_des_added[v1_sci]['dec'],place)))

    try:
        inds.append(zipped_locs.index((targ[0],targ[1])))
        inds_new.append(zipped_locs_new.index((targ[0],targ[1])))
    except ValueError:
        print("not in list")

    #print(i[0], i[1])
v05_scionly = old_des_remaining[v05_sci]
v05_scionly.remove_rows(inds)
v1_scionly = new_des_added[v1_sci]
v1_scionly.remove_rows(inds_new)

In [69]:
tol = 3/3600
print(len(v1_scionly))
issues = []
iss_counter = 0
for i in v1_scionly:
    ra = ((v05_scionly['ra'] <= (i['ra'] +tol))&(v05_scionly['ra'] >= (i['ra'] -tol)))
    dec = ((v05_scionly['dec'] <= (i['dec'] +tol))&(v05_scionly['dec'] >= (i['dec'] -tol)))
    find_it = np.where(ra&dec)[0]
    
    print(i['ra', 'dec', 'priority'])
    print(v05_scionly['ra', 'dec', 'priority'][find_it])
    
    try:
        lowest = np.argmin(v05_scionly['priority'][find_it])
        print("new priority: ", v05_scionly['priority'][find_it][lowest])
        print("priority reset!")
        rmV1['priority'][np.where(rmV1['ra'] == i['ra'])] = v05_scionly['priority'][find_it][lowest]
    except ValueError:
        print("--"*100)
        print("no match?")
        issues.append((i['ra'],i[ 'dec']))
        iss_counter += 1
        print("--"*100)

893
   ra      dec    priority
-------- -------- --------
52.13623 -28.7335   1037.0
    ra       dec     priority
--------- ---------- --------
52.136227 -28.733494   1037.0
52.136227 -28.733494   1500.0
new priority:  1037.0
priority reset!
    ra       dec     priority
--------- ---------- --------
53.392185 -27.573502   1047.0
   ra      dec     priority
------- ---------- --------
53.3922 -27.573496   1047.0
53.3922 -27.573496   1500.0
53.3922 -27.573496   1500.0
new priority:  1047.0
priority reset!
   ra       dec     priority
-------- ---------- --------
53.15991 -27.361511   1047.0
   ra       dec     priority
-------- ---------- --------
53.15995 -27.361465   1046.0
53.15995 -27.361465   1047.0
new priority:  1046.0
priority reset!
    ra       dec     priority
--------- ---------- --------
51.973927 -28.626791   1041.0
    ra       dec     priority
--------- ---------- --------
51.973927 -28.626791    942.0
new priority:  942.0
priority reset!
    ra       dec     priority
-

    ra       dec     priority
--------- ---------- --------
53.008076 -28.668915   1046.0
 ra dec priority
--- --- --------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
no match?
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    ra       dec     priority
--------- ---------- --------
53.458084 -28.676622   1041.0
 ra dec priority
--- --- --------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
no match?
---------------------------------------------------------------------------------------------------------------------------------

    ra       dec     priority
--------- ---------- --------
53.276806 -28.235554   1033.0
    ra       dec     priority
--------- ---------- --------
53.276806 -28.235554   1033.0
53.276806 -28.235554   1034.0
new priority:  1033.0
priority reset!
    ra       dec     priority
--------- ---------- --------
53.414043 -28.290092   1033.0
    ra       dec     priority
--------- ---------- --------
53.414043 -28.290092    932.0
53.414043 -28.290092    933.0
53.414043 -28.290092    934.0
new priority:  932.0
priority reset!
    ra       dec     priority
--------- ---------- --------
53.133373 -28.052795   1033.0
    ra       dec     priority
--------- ---------- --------
53.133373 -28.052795    932.0
53.133373 -28.052795    933.0
53.133373 -28.052795    934.0
53.133373 -28.052795   1500.0
53.133373 -28.052795   1500.0
new priority:  932.0
priority reset!
   ra       dec    priority
-------- --------- --------
53.29049 -27.93729   1038.0
   ra       dec    priority
-------- --------- -------

    ra       dec     priority
--------- ---------- --------
52.483147 -28.160677   1039.0
    ra       dec     priority
--------- ---------- --------
52.483147 -28.160677    939.0
new priority:  939.0
priority reset!
   ra       dec     priority
-------- ---------- --------
52.16498 -27.883501   1044.0
 ra dec priority
--- --- --------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
no match?
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    ra       dec     priority
--------- ---------- --------
53.114033 -28.097689   1029.0
   ra       dec     priority
-------- ---------- --------
53.11403 -28.097687   1050.0
53.11403 -28.097687   1050.0
new priority:  1050.0
priority reset!
    ra

   ra       dec     priority
-------- ---------- --------
53.08099 -27.350842   1521.0
 ra dec priority
--- --- --------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
no match?
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    ra       dec     priority
--------- ---------- --------
53.071537 -27.332355   1521.0
    ra       dec     priority
--------- ---------- --------
53.071537 -27.332355    922.0
53.071537 -27.332355    923.0
53.071537 -27.332355    924.0
53.071537 -27.332355   1500.0
new priority:  922.0
priority reset!
    ra       dec    priority
--------- --------- --------
53.010242 -27.27183   1520.0
    ra       dec    priority
--------- --------- --------
53.010246 -27.2

In [70]:
iss_counter

205

In [71]:
#let's do a quick and dirty -50 boost on previously observed targets

# targ_lists = glob.glob("*_sciencetargs.dat")
# del targ_lists[1]
# del targ_lists[2]
# del targ_lists[4]

fields = {"RM":'./104667_sciencetargs.dat',
          "XMMLSS":'./104669_sciencetargs.dat', 
          "COSMOS": './104668_sciencetargs.dat',
          "CDFS": './CDFS_sciencetargs.dat'}

#CHANGE FOR EACH FIELD
field = 'CDFS'

prev_targeted = Table.read(fields[field], format='ascii')
tol = 3/3600
for i in prev_targeted:
    #print(i['ra'], i['dec'])
    RA_mask = ((rmV1['ra'] <= (i['ra'] +tol))&(rmV1['ra'] >= (i['ra'] -tol)))
    Dec_mask = ((rmV1['dec'] <= (i['dec'] +tol))&(rmV1['dec'] >= (i['dec'] -tol)))
    targ = np.where(RA_mask&Dec_mask)[0]
    
    print(rmV1['ra', 'dec','priority'][targ])
    rmV1['priority'][targ] -= 50
    print("new:", rmV1['priority'][targ])


   ra      dec     priority
------- ---------- --------
52.6275 -28.359852   1500.0
52.6275 -28.359852   1500.0
new: priority
--------
  1450.0
  1450.0
    ra       dec     priority
--------- ---------- --------
53.127644 -28.797346    937.0
53.127644 -28.797346    937.0
53.127644 -28.797346    937.0
53.127644 -28.797346    937.0
53.127644 -28.797346    937.0
new: priority
--------
   887.0
   887.0
   887.0
   887.0
   887.0
  ra      dec     priority
------ ---------- --------
53.471 -28.506224    917.0
53.471 -28.506224    917.0
53.471 -28.506224    917.0
53.471 -28.506224    917.0
53.471 -28.506224    917.0
new: priority
--------
   867.0
   867.0
   867.0
   867.0
   867.0
    ra       dec     priority
--------- ---------- --------
53.304604 -28.671665    933.0
53.304604 -28.671665    933.0
53.304604 -28.671665    933.0
53.304604 -28.671665    933.0
new: priority
--------
   883.0
   883.0
   883.0
   883.0
   ra       dec     priority
-------- ---------- --------
53.24847 -28.58

   ra       dec     priority
-------- ---------- --------
53.07855 -27.758205   1001.0
new: priority
--------
   951.0
    ra       dec     priority
--------- ---------- --------
53.074528 -27.849874   1001.0
new: priority
--------
   951.0
   ra       dec     priority
-------- ---------- --------
53.56311 -27.858728   1001.0
new: priority
--------
   951.0
    ra       dec    priority
--------- --------- --------
53.549458 -27.80063    901.0
53.549458 -27.80063    901.0
new: priority
--------
   851.0
   851.0
    ra       dec     priority
--------- ---------- --------
52.933865 -27.852163   1001.0
new: priority
--------
   951.0
   ra       dec     priority
-------- ---------- --------
53.07114 -27.822763   1500.0
53.07114 -27.822763   1500.0
new: priority
--------
  1450.0
  1450.0
    ra       dec     priority
--------- ---------- --------
53.113293 -27.737999   1001.0
new: priority
--------
   951.0
    ra       dec     priority
--------- ---------- --------
52.945602 -27.416895  

In [65]:
rmV1.write("BHM_cartons_{}_v1_newpriorities.fits".format(field), format='fits', overwrite=True)